In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16


2023-11-28 17:05:22.067783: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 17:05:22.067847: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 17:05:22.067929: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 17:05:22.085322: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt


In [8]:
dataset_path = './img/'
batch_size = 8 
transform = transforms.Compose([
    #transforms.Resize((80, 80)),
    transforms.ToTensor(),
])

train_dataset = ImageFolder(root='./data_split/train', transform=transform)
val_dataset = ImageFolder(root='./data_split/test', transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [9]:
class CustomFCNetwork(nn.Module):
    def __init__(self):
        super(CustomFCNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(19200, 128)  # Update fc1 input size to match the expected size
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 60)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(60, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

class LeNet5V1(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature = nn.Sequential(
            # 1
            nn.Conv2d(in_channels=3,    # cantidad de canales RGB == 3
                      out_channels=6, 
                      kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),

            # 2
            nn.Conv2d(in_channels=6, 
                      out_channels=16,
                      kernel_size=5, stride=1),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=16*22*22, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=10),
            # nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.classifier(self.feature(x))

In [10]:
try:
    modelCNN = LeNet5V1()
    model_name = "LeNet5_final"
    modelCNN.load_state_dict(torch.load(f"models/best_model_{model_name}.pth", map_location=torch.device('cpu')))
    modelCNN.eval()  # Set model to evaluation mode after loading
except Exception as e:
    print(f"Error: {e}")


In [11]:
try:
    modelFCN = CustomFCNetwork()
    model_name = "FCN_final"
    modelFCN.load_state_dict(torch.load(f"models/best_model_{model_name}.pth", map_location=torch.device('cpu')))
    modelFCN.eval()  # Set model to evaluation mode after loading
except Exception as e:
    print(f"Error: {e}")


In [23]:
import os
import torch
import numpy as np
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision.utils import save_image


In [ ]:
path = 'data_split/test/'

fcn_model = load_model("./models/" + f'adversarial_FCN.pth', CustomFCNetwork())


In [26]:
path = 'data_split/test/'

cnn_model = load_model("./models/" + f'adversarial_LeNet5.pth', LeNet5V1())


TypeError: 'LeNet5V1' object is not a mapping

In [1]:

num_classes = 5

def synthesize_stimuli(model, target_activation, original_image, epsilon=1e-2, num_iterations=100):

    model.eval()
    original_image = original_image.clone().detach().requires_grad_(True)
    
    optimizer = torch.optim.SGD([original_image], lr=epsilon)

    for _ in range(num_iterations):
        optimizer.zero_grad()
        
        # Forward pass
        output = model(original_image)
        target_layer_activation = model.get_penultimate_activation(original_image)
        
        # Loss function: Diferencia cuadrática entre activaciones
        loss = nn.MSELoss()(target_layer_activation, target_activation)
        
        # Backward pass
        loss.backward()
        
        # Update image
        optimizer.step()

    return original_image.detach()

# Función para generar los gráficos solicitados
def generate_plots(original_images, synthesized_cnn_images, synthesized_fcn_images, num_classes):
    fig, axes = plt.subplots(num_classes, 3, figsize=(10, 3 * num_classes))

    for i in range(num_classes):
        # Columna 1: Original
        axes[i, 0].imshow(original_images[i].permute(1, 2, 0))
        axes[i, 0].axis('off')
        axes[i, 0].set_title(f"Class {i + 1}\nOriginal")

        # Columna 2: Synthesized CNN
        axes[i, 1].imshow(synthesized_cnn_images[i].permute(1, 2, 0))
        axes[i, 1].axis('off')
        axes[i, 1].set_title(f"Synthesized CNN\nClass {i + 1}")

        # Columna 3: Synthesized Ftarget_activation_cnnCN
        axes[i, 2].imshow(synthesized_fcn_images[i].permute(1, 2, 0))
        axes[i, 2].axis('off')
        axes[i, 2].set_title(f"Synthesized FCN\nClass {i + 1}")

    plt.tight_layout()
    plt.show()

# Obtener imágenes originales de cada clase
original_images = [train_dataset[i][0] for i in range(num_classes)]

# Sintetizar estímulos coincidentes para cada red (CNN y FCN)
synthesized_cnn_images = []
synthesized_fcn_images = []

for i in range(num_classes):
    target_activation_cnn = modelCNN.get_penultimate_activation(original_images[i])
    synthesized_cnn = synthesize_stimuli(modelCNN, target_activation_cnn, original_images[i])
    synthesized_cnn_images.append(synthesized_cnn)

    target_activation_fcn = modelFCN.get_penultimate_activation(original_images[i])
    synthesized_fcn = synthesize_stimuli(modelFCN, target_activation_fcn, original_images[i])
    synthesized_fcn_images.append(synthesized_fcn)

# Generar y mostrar los gráficos
generate_plots(original_images, synthesized_cnn_images, synthesized_fcn_images, num_classes)

NameError: name 'train_dataset' is not defined